In [1]:
import itertools
import random

import tensorflow as tf
import os
from itertools import chain
from pathlib import Path
import matplotlib.pyplot as plt
import threading

2022-12-16 12:13:37.661952: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-16 12:13:38.106300: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-16 12:13:38.106329: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-16 12:13:39.892519: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
path:Path = Path("tinyface/Training_Set")
paths = [Path(path/i) for i in os.listdir(path)]
all_samples= [[Path(i/image_path).as_posix() for image_path in os.listdir(i)] for i in paths]
hom_many_people = len(all_samples)

In [3]:
all_samples_combined = []
all_samples_product = []

for i in range(len(all_samples)):
    for j in all_samples[0:i]+all_samples[i+1:]:
        all_samples_combined.append((all_samples[i],all_samples[i],j))

random.shuffle(all_samples_combined)

mutex = threading.Semaphore(1)
bazming = True
def permutation(i):
    if i/len(all_samples_combined)>0.05 :
        bazming = False
        return
    global all_samples_product
    res = list(itertools.product(*(all_samples_combined[i])))
    for item in res:
        if item[0] == item[1]:
            res.remove(item)
    if i%1000 == 0 :
        print(i/len(all_samples_combined))
    mutex.acquire()
    all_samples_product+= res
    mutex.release()



thread_handles = []
for i in range(len(all_samples_combined)):
    if bazming is not True:
        break
    a = threading.Thread(target=permutation, args=(i,))
    a.start()
    thread_handles.append(a)


0.0
0.00015146168095202755
0.0003029233619040551
0.00045438504285608264
0.0006058467238081102
0.0007573084047601377
0.0009087700857121653
0.0010602317666641929
0.0012116934476162204
0.0013631551285682479
0.0015146168095202753
0.001666078490472303
0.0018175401714243305
0.001969001852376358
0.0021204635333283857
0.002271925214280413
0.0024233868952324407
0.002574848576184468
0.0027263102571364957
0.0028777719380885234
0.0030292336190405507
0.0031806952999925784
0.003332156980944606
0.0034836186618966334
0.003635080342848661
0.0037865420238006884
0.003938003704752716
0.004089465385704744
0.0042409270666567715
0.004392388747608798
0.004543850428560826
0.004695312109512854
0.0048467737904648815
0.004998235471416909
0.005149697152368936
0.005301158833320964
0.005452620514272991
0.005604082195225019
0.005755543876177047
0.0059070055571290745
0.006058467238081101
0.006209928919033129
0.006361390599985157
0.0065128522809371845
0.006664313961889212
0.006815775642841239
0.006967237323793267
0.007

KeyboardInterrupt: 

In [4]:
for i in thread_handles:
    i.join()

In [5]:
thread_handles = []

paths = []
all_samples = []
all_samples_combined = []
all_samples_product = all_samples_product[0:int(4708924*2/4)]

In [6]:
len(all_samples_product)

2354462

In [7]:

list_ds =  tf.data.Dataset.from_tensor_slices(all_samples_product)
list_ds = list_ds.shuffle(len(list_ds), reshuffle_each_iteration=False)

2022-12-16 12:15:07.155163: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-16 12:15:07.155224: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dimo-81ne): /proc/driver/nvidia/version does not exist
2022-12-16 12:15:07.157673: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
val_size = int(len(list_ds) * 0.2)
train_ds = list_ds.skip(val_size)
val_ds = list_ds.take(val_size)


In [9]:
AUTOTUNE = tf.data.AUTOTUNE
img_dims = [32,32,3]
batch_size = 64
shuffle_buffer_size = 1000

In [10]:

def decode_img(img):
    # Convert the compressed string to a 3D uint8 tensor
    img = tf.io.decode_jpeg(img, channels=img_dims[2])
    img = tf.image.convert_image_dtype(img, tf.float32)
    # Resize the image to the desired size
    return tf.image.resize(img, img_dims[:2])


def process_path(file_path):
    # Load the raw data from the file as a string
    img:tf.Tensor = tf.io.read_file(file_path)
    img = decode_img(img)
    return img

def process_triplets(file_items0):
    return process_path(file_items0[0]),process_path(file_items0[1]),process_path(file_items0[2])

def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds



In [11]:

train_ds = configure_for_performance(train_ds.map(process_triplets, num_parallel_calls=AUTOTUNE))
val_ds = configure_for_performance(val_ds.map(process_triplets, num_parallel_calls=AUTOTUNE))

In [12]:
k = train_ds.take(1)
print(k)

<TakeDataset element_spec=(TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None))>


In [13]:
from keras import layers

def image_encode_creator():
    model = tf.keras.Sequential([
        layers.Conv2D(8, 3,  activation='relu'),
        layers.MaxPool2D(),
        layers.Conv2D(8,  3, activation='relu'),
        layers.MaxPool2D(),
        layers.Flatten(),
        layers.Dense(256,activation="relu")
    ])
    return model


Encoder = image_encode_creator()


In [14]:
class DistanceLayer(tf.keras.layers.Layer):
    """
    This layer is responsible for computing the distance between the anchor
    embedding and the positive embedding, and the anchor embedding and the
    negative embedding.
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return ap_distance, an_distance


In [15]:

anchor_input = tf.keras.layers.Input(shape=(32, 32, 3))
positive_input = tf.keras.layers.Input(shape=(32, 32, 3))
negative_input = tf.keras.layers.Input(shape=(32, 32, 3))

distances = DistanceLayer()(
    Encoder(anchor_input),
    Encoder(positive_input),
    Encoder(negative_input),
)

siamese_network = tf.keras.Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances
)

In [16]:
class SiameseModel(tf.keras.Model):
    """The Siamese Network model with a custom training and testing loops.

    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.

    The triplet loss is defined as:
       L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    """

    def __init__(self, network, margin=0.5):
        super(SiameseModel, self).__init__()
        self.siamese_network = siamese_network
        self.margin = margin
        self.loss_tracker = tf.keras.metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape is a context manager that records every operation that
        # you do inside. We are using it here to compute the loss so we can get
        # the gradients and apply them using the optimizer specified in
        # `compile()`.
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)

        # Storing the gradients of the loss function with respect to the
        # weights/parameters.
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)

        # Applying the gradients on the model using the specified optimizer
        self.optimizer.apply_gradients(
            zip(gradients, self.siamese_network.trainable_weights)
        )

        # Let's update and return the training loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)

        # Let's update and return the loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        # The output of the network is a tuple containing the distances
        # between the anchor and the positive example, and the anchor and
        # the negative example.
        ap_distance, an_distance = self.siamese_network(data)

        # Computing the Triplet Loss by subtracting both distances and
        # making sure we don't get a negative value.
        loss = ap_distance - an_distance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker]


In [ ]:
siamese_model = SiameseModel(siamese_network)
siamese_model.compile(optimizer=tf.keras.optimizers.Adam(0.0001))
siamese_model.fit(train_ds, epochs=10, validation_data=val_ds)


Epoch 1/10
 2145/29431 [=>............................] - ETA: 30:48 - loss: 0.1516